## Distributed Training with Chainer and ChainerMN

Chainer can train in two modes: single-machine, and distributed. Unlike the single-machine notebook example that trains an image classification model on the CIFAR-10 dataset, we will write a Chainer script that uses `chainermn` to distribute training to multiple instances.

[VGG](https://arxiv.org/pdf/1409.1556v6.pdf) is an architecture for deep convolution networks. In this example, we train a convolutional network to perform image classification using the CIFAR-10 dataset on multiple instances. CIFAR-10 consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. We'll train a model on SageMaker, deploy it to Amazon SageMaker, and then classify images using the deployed model.

The Chainer script runs inside of a Docker container running on SageMaker. For more on the Chainer container, please visit the sagemaker-chainer-containers repository and the sagemaker-python-sdk repository:

* https://github.com/aws/sagemaker-chainer-containers
* https://github.com/aws/sagemaker-python-sdk

For more on Chainer and ChainerMN, please visit the Chainer and ChainerMN repositories:

* https://github.com/chainer/chainer
* https://github.com/chainer/chainermn

This notebook is adapted from the [CIFAR-10](https://github.com/chainer/chainer/tree/master/examples/cifar) example in the Chainer repository.

In [ ]:
# Setup
from sagemaker import get_execution_role
import sagemaker

sagemaker_session = sagemaker.Session()

# This role retrieves the SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()
help(sagemaker_session.upload_data)

## Downloading training and test data

We use helper functions given by `chainer` to download and preprocess the CIFAR10 data. 

In [ ]:
import chainer

from chainer.datasets import get_cifar10

train, test = get_cifar10()

## Uploading the data

We save the preprocessed data to the local filesystem, and then use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value `inputs` identifies the S3 location, which we will use when we start the Training Job.

In [ ]:
import os
import shutil

import numpy as np

train_data = [element[0] for element in train]
train_labels = [element[1] for element in train]

test_data = [element[0] for element in test]
test_labels = [element[1] for element in test]


try:
    os.makedirs('/tmp/data/distributed_train_cifar')
    os.makedirs('/tmp/data/distributed_test_cifar')
    np.savez('/tmp/data/distributed_train_cifar/train.npz', data=train_data, labels=train_labels)
    np.savez('/tmp/data/distributed_test_cifar/test.npz', data=test_data, labels=test_labels)
    train_input = sagemaker_session.upload_data(path=os.path.join('/tmp', 'data', 'distributed_train_cifar'),
                                                            key_prefix='notebook/distributed_chainer_cifar/train')
    test_input = sagemaker_session.upload_data(path=os.path.join('/tmp', 'data', 'distributed_test_cifar'),
                                                           key_prefix='notebook/distributed_chainer_cifar/test')
finally:
    shutil.rmtree('/tmp/data')
print('training data at ', train_input)
print('test data at ', test_input)

## Writing the Chainer script to run on Amazon SageMaker

### Training

We need to provide a training script that can run on the SageMaker platform. The training script is very similar to a training script you might run outside of SageMaker, but you can construct a `sagemaker_containers.env.TrainingEnv` instance to discover useful properties from the training environment, such as:

  * `training_env.model_dir (str)`: path to the directory to write model artifacts to. These artifacts are uploaded to S3 for model hosting.
  * `training_env.num_gpus (int): ` The number of GPUs available to the host.
  * `training_env.channel_input_dirs (dict of str: str)`: A map of input channel names (like 'train' and 'test') to filesystem paths to data in those input channels. 
  * `training_env.output_data_dir (str)`: The filesystem path to write output artifacts to. Output artifacts may include checkpoints, graphs, and other files to save, not including model artifacts. These artifacts are compressed and uploaded to S3 to the same S3 prefix as the model artifacts.

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an `argparse.ArgumentParser` instance.

Because the Chainer container imports your training script, you should always put your training code in a main guard (`if __name__=='__main__':`) so that the container does not inadvertently run your training code at the wrong point in execution.

For more on `TrainingEnv`, please visit https://github.com/aws/sagemaker-containers.

### Hosting and Inference

We will use the same script to host the Chainer model as we will use to train it (but this is not necessary -- we could use separate training and hosting scripts). In contrast with the training script, the hosting script requires you to implement functions with particular function signatures (or rely on defaults for those functions).

These function hooks load your model, deserialize data sent by a client, obtain inferences from your loaded model, and serialize predictions back to a client:

* **`model_fn(model_dir)`**: This function is invoked to load model artifacts from those written into `model_dir` during training.
* `input_fn(input_data, content_type)`: This function is invoked to deserialize prediction data when a prediction request is made. The return value is passed to predict_fn. `input_fn` accepts two arguments: `input_data`, which is the serialized input data in the body of the prediction request, and `content_type`, the MIME type of the data
  
  
* `predict_fn(input_data, model)`: This function accepts the return value of `input_fn` (as `input_data`) and the return value of `model_fn`, `model`, and returns inferences obtained from the model
  
  
* `output_fn(prediction, accept)`: This function is invoked to serialize the return value from `predict_fn`, passed in via `prediction`, back to the SageMaker client in response to prediction requests


`model_fn` is always required, but defaults exist for the remaining functions. These defaults can deserialize a NumPy array, invoking the model's `__call__` method on the input data, and serialize a NumPy array back to the client.

This notebook relies on the default `input_fn`, `predict_fn`, and `output_fn`. See the Chainer sentiment analysis notebook for an example of how one can implement these hosting functions.

Please examine the script below. Training occurs behind the main guard, and `model_fn` loads the model saved into `model_dir` during training.

The script below uses a chainermn Communicator to distribute training to multiple nodes. The Communicator depends on MPI (Message Passing Interface), so the Chainer container running on SageMaker runs this script with mpirun if the Chainer Estimator specifies a train_instance_count of two or greater, or if use_mpi in the Chainer estimator is true.

By default, one process is created per GPU (on GPU instances), or one per host (on CPU instances, which are not recommended for this notebook).

For more on writing Chainer scripts to run on SageMaker, or for more on the Chainer container itself, please see the following repositories: 

* For writing Chainer scripts to run on SageMaker: https://github.com/aws/sagemaker-python-sdk
* For more on the Chainer container and default hosting functions: https://github.com/aws/sagemaker-chainer-containers


In [ ]:
!cat 'src/chainer_cifar_vgg_distributed.py'

## Running the training script on SageMaker

To train with a Chainer script, we construct a ```Chainer``` estimator using the [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk). We pass in an `entry_point`, the name of a script that contains a couple of functions with certain signatures (`train` and `model_fn`), and a `source_dir`, a directory containing all code to run inside the Chainer container. This script will be run on SageMaker in a container that invokes these functions to train and load Chainer models. 

The ```Chainer``` class allows us to run our training function as a training job on SageMaker infrastructure. We need to configure it with our training script, an IAM role, the number of training instances, and the training instance type. In this case we will run our training job on two `ml.p3.2xlarge` instances.

This script uses the `chainermn` package, which distributes training with MPI. Your script is run with `mpirun`, so a ChainerMN Communicator object can be used to distribute training. Arguments to `mpirun` are set to sensible defaults, but you can configure how your script is run in distributed mode. See the ```Chainer``` class documentation for more on configuring MPI.

In [ ]:
from sagemaker.chainer.estimator import Chainer

chainer_estimator = Chainer(entry_point='chainer_cifar_vgg_distributed.py', source_dir="src", role=role,
                            use_mpi=True, sagemaker_session=sagemaker_session,
                            train_instance_count=2, train_instance_type='ml.p3.2xlarge',
                            hyperparameters={'epochs': 30, 'batch-size': 256})

chainer_estimator.fit({'train': train_input, 'test': test_input})

Our Chainer script writes various artifacts, such as plots, to a directory `output_data_dir`, the contents of which which SageMaker uploads to S3. Now we download and extract these artifacts.

In [ ]:
from s3_util import retrieve_output_from_s3

chainer_training_job = chainer_estimator.latest_training_job.name

desc = sagemaker_session.sagemaker_client.describe_training_job(TrainingJobName=chainer_training_job)
output_data = desc['ModelArtifacts']['S3ModelArtifacts'].replace('model.tar.gz', 'output.tar.gz')

retrieve_output_from_s3(output_data, 'output/distributed_cifar')

These plots show the accuracy and loss over epochs:

In [ ]:
from IPython.display import Image
from IPython.display import display

accuracy_graph = Image(filename = "output/distributed_cifar/algo-1/accuracy.png", width=800, height=800)
loss_graph = Image(filename = "output/distributed_cifar/algo-1/loss.png", width=800, height=800)

display(accuracy_graph, loss_graph)

## Deploying the Trained Model

After training, we use the Chainer estimator object to create and deploy a hosted prediction endpoint. We can use a CPU-based instance for inference (in this case an `ml.m4.xlarge`), even though we trained on GPU instances.

The predictor object returned by `deploy` lets us call the new endpoint and perform inference on our sample images. 

In [ ]:
predictor = chainer_estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

### CIFAR10 sample images

We'll use these CIFAR10 sample images to test the service:

<img style="display: inline; height: 32px; margin: 0.25em" src="images/airplane1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/automobile1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/bird1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/cat1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/deer1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/dog1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/frog1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/horse1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/ship1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/truck1.png" />



## Predicting using SageMaker Endpoint

We batch the images together into a single NumPy array to obtain multiple inferences with a single prediction request.

In [ ]:
from skimage import io
import numpy as np

def read_image(filename):
    img = io.imread(filename)
    img = np.array(img).transpose(2, 0, 1)
    img = np.expand_dims(img, axis=0)
    img = img.astype(np.float32)
    img *= 1. / 255.
    img = img.reshape(3, 32, 32)
    return img


def read_images(filenames):
    return np.array([read_image(f) for f in filenames])

filenames = ['images/airplane1.png',
             'images/automobile1.png',
             'images/bird1.png',
             'images/cat1.png',
             'images/deer1.png',
             'images/dog1.png',
             'images/frog1.png',
             'images/horse1.png',
             'images/ship1.png',
             'images/truck1.png']

image_data = read_images(filenames)

The predictor runs inference on our input data and returns a list of predictions whose argmax gives the predicted label of the input data. 

In [ ]:
response = predictor.predict(image_data)

for i, prediction in enumerate(response):
    print('image {}: prediction: {}'.format(i, prediction.argmax(axis=0)))

## Cleanup

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it.

In [ ]:
chainer_estimator.delete_endpoint()